In [4]:
!pip install numpy
!pip install panda


[notice] A new release of pip available: 22.3.1 -> 23.0
[notice] To update, run: python -m pip install --upgrade pip


In [7]:
import pandas as pd
import numpy as np


In [9]:
df_train=pd.read_csv('data/train.csv')

/tmp/ipykernel_16229/3747062319.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df_train=pd.read_csv('data/train.csv')
